<center> <h1> <b> Data Science på Flashback </b> <br> ...i dubbel bemärkelse </h1> </center>

Daniel Westerlund <br>
westerlund@gmail.com <br>
2021-08-15

* [Introduktion](#Introduktion)
* [Datahämtning](#Datahämtning)
    - [Hur ska data hämtas?](#Hur-ska-data-hämtas?)
    - [Webbskrapning](#Webbskrapning)
* [Analys av data](#Analys-av-data)
    - [Trådanalys](#Trådanalys)
    - [Ordanalys](#Ordanalys)
* [Slutsats](#Slutsats)
* [Källförteckning](#Källförteckning)

# Introduktion

Data Science som karriärspår är något många på kort tid har börjat visa stort intresse för. Har man läst artiklar på nätet om dess historia vet man att termen har funnits i årtionden (testa att googla “when did data science become a thing”), men min uppfattning är att Data Science i dess nutida mening inte tog fart förrän 2010-talet, i takt med att exempelvis smart phones och maskininlärning gjorde entré i världen.

I det här projektet har jag tänkt att utforska antalet trådstartar på Sveriges största webbforum (Flashback) som innehåller vissa nyckelord eller nyckelfraser, till exempel “deep learning”, “big data”, “maskininlärning” eller givetvis “data scientist”. För att filtrera bort så många trådar som möjligt på förhand kommer jag hålla mig till två utvalda kategorier, nämligen *Utbildning och studier* samt *Arbetsliv och arbetsmarknad*. Målet är att visualisera hur “kvantiteten diskussion” kring data science har utvecklats under 2010-talet på webben och på så sätt ge en bild av hur intresset för ämnet har utvecklats IRL. Tesen är att en uppåtlutande trend kommer avslöja sig.

Utöver detta ska jag även undersöka vilka de vanligaste orden i dessa inlägg är, samt analysera om den samlade styrkan i orden pekar på att inläggen är positiva eller negativa.

# Datahämtning

## Hur ska data hämtas?

Det finns för mig ingen tillgång till någon databas eller något API att hämta data från. Istället kommer webbskrapning att utnyttjas. Det innebär att jag kommer använda en plug-in som heter *SelectorGadget* för att på ett dynamiskt och effektivt sätt hämta önskad data från sajten. Det finns andra sätt att skrapa webbsidor på också, som copy-paste eller text pattern matching, men båda dessa alternativ skulle ta längre tid samtidigt som de lämnar större utrymme åt felaktigheter under inhämtningen av data. 

## Webbskrapning

Alla kommentarer, variabler och argument låter jag vara på engelska. Det blir enklare för läsaren att förstå koden om texten och koden har ett bra flyt, vilket underlättas av att språket är engelska och inte "svengelska". 

Låt oss börja med att skapa en funktion `get_basics` för att hämta sex grundläggande variabler, nämligen

* *title*: ämnesrubrik
* *last_response*: datum för senaste svaret
* *responses*: antal svar
* *views*: antal visningar
* *link*: länk till tråden
* *categorie*: kategorin som tråden tillhör.

Denna funktion returnerar en data frame (tabell) med dessa variabler, och kommer kallas på från en for-loop 2 gånger 647 gånger. Vi vill nämligen hämta trådar från så många sidor som möjligt, men från två olika kategorier. Eftersom det i skrivande stund "endast" finns 647 sidor för kategorin *Utbildning och studier* låter vi 647 vara antalet sidor per kategori. 


In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re
import time

In [ ]:

def get_basics():
    ''' Collect data, return 2D array '''
    data_entries = np.empty((0, 6), str)
    categories_dict = {'Utbildning och studier': 'https://www.flashback.org/f303', 
                        'Arbetsliv och arbetsmarknad': 'https://www.flashback.org/f105'}
    for categorie, threads_link in categories_dict.items():
        for page_num in range(1, 2):
            time.sleep(1)
            thread_table_link = threads_link + 'p' + str(page_num)
            html_text = requests.get(thread_table_link, headers={"User-Agent":"Chrome/78.0.3904.87"}).text
            soup = BeautifulSoup(html_text, 'lxml')
            threads = soup.find('table', class_ = 'table table-striped table-threads table-condensed') \
                                .find('tbody').find_all('tr', class_ = None)
            for thread in threads:
                title = thread.find('td', class_='td_title').find('a').text
                last_response = thread.find('td', class_ = 'td_last_post text-right nowrap hidden-xs').find('div').text.strip()

                replies_views = thread.find('td', class_ = 'td_replies nowrap text-right hidden-xs').find_all('div')
                responses = ''.join(re.findall('\d+', replies_views[0].text))
                views = ''.join(re.findall('\d+', replies_views[1].text))

                link = 'https://www.flashback.org' + thread.find('td', class_='td_title').find('a')['href']

                # Add data to the data_entries array
                row_data = np.array([[title, last_response, responses, views, link, categorie]])
                data_entries = np.append(arr=data_entries, values=row_data, axis=0)
    return data_entries

flashback_data = get_basics()

flashback_df = pd.DataFrame(data=flashback_data, columns=['title', 'last_response', 'responses', 'views', 'link', 'categorie'])

In [25]:
# Save flashback_df locally 
flashback_df.to_csv('flashback_df.csv', index=False, encoding='windows-1252', sep=';')

In [6]:
# Load flashback_df from local storage so we don't have to re-scrape the web
flashback_df = pd.read_csv('flashback_df.csv', encoding='windows-1252', sep=';')
flashback_df

,title,last_response,responses,views,link,categorie
0,Tråden om HHS,Idag 13:53,9047,1735923,https://www.flashback.org/t627392,Utbildning och studier
1,CSN lån för CFA level 1?,Idag 13:24,7,723,https://www.flashback.org/t3335450,Utbildning och studier
2,Dotter börjar i klass med bara elever som inte...,Idag 12:32,210,31315,https://www.flashback.org/t3345746,Utbildning och studier
3,Dags för en utbildning men vilken?,Igår 22:23,4,570,https://www.flashback.org/t3344803,Utbildning och studier
4,"Ni som läste en ""värdelös"" utbildning på unive...",Igår 20:37,119,26524,https://www.flashback.org/t3343951,Utbildning och studier
...,...,...,...,...,...,...
95,Plåtslagare eller Lastbilsmekaniker i högre ålder,2021-08-03 17:10,10,475,https://www.flashback.org/t3345628,Arbetsliv och arbetsmarknad
96,Hur lång tid innan Svenskt Näringsliv är ute o...,2021-08-03 13:39,380,18463,https://www.flashback.org/t3136899,Arbetsliv och arbetsmarknad
97,Går det att säga upp någon som har sjukfrånvar...,2021-08-03 13:10,80,8705,https://www.flashback.org/t3340455,Arbetsliv och arbetsmarknad
98,Löneläge för Fastighetsförvaltare & liknande t...,2021-08-03 13:00,23,3884,https://www.flashback.org/t3071058,Arbetsliv och arbetsmarknad


Innan vi går vidare till att hämta sista bitarna information från sajten måste vi filtrera bort alla trådar som inte är av intresse. Den filtrerade version låter vi heta `filtered_df`.

`flashback_df` filtreras med funktionen `filter` genom att endast behålla de rader vars titel innehåller något av orden i vektorn `keywords`. Orden hämtas från filen nyckelord.csv, en .csv-fil innehållande 31 stycken ord som relaterar till Data Science. 

Efter filtreringen innehåller `filtered_df` precis samma variabler som `flashback_df`, men endast trådar som relaterar till ämnet är kvar. Bra start, men vi vill ha mer information för att kunna utföra analysen. Datum för trådstart hämtas genom att iterera genom `filtered_df` och använda nedanstående funktion `get_date` samt dess hjälpfunktion `date_converter` för de fall då en tråd startats idag eller igår. Variabeln *last_response*, alltså datum för senaste svaret i respektive tråd, konverteras till Date-format med hjälp av `date_converter` och `as.Date`.

In [23]:
# Filter flashback_df for titles containing at least one of the words from keywords.csv
import csv

with open('nyckelord.csv', newline='') as f:
    reader = csv.reader(f)
    unflattened_wordlist = list(reader)
    keywords = [word + '|' for sublist in unflattened_wordlist[0:-1] for word in sublist]
    keywords_string = ''.join(keywords) + unflattened_wordlist[-1][0]

flashback_df = flashback_df[flashback_df['title'].str.contains(keywords_string)]

Endast en sista variabel ska hämtas, nämligen varje tråds första inlägg. Detta är alltså trådstartarens öppningsinlägg och ska inte förväxlas med det första svaret, som ju är inlägg nummer två (denna hämtar vi inte). Inhämtningen sker likt datuminhämtningen ovan. Den slutgiltiga data frame:en har nu följande struktur:

In [4]:
# Glimpse of data frame

# Analys av data

## Trådanalys

Inledningsvis är det en god idé att skaffa sig en enkel dataöverblick. Till exemepel är andelen trådar från respektive kategori 17% från Arbetsliv och arbetsmarknad, och resterande 83% är från Utbildning och studier.

In [5]:
# Diagram: Percentages of threads per categorie 

Om vi tar alla dessa trådar och dividerar antalet med totala antalet trådar från `flashback_df`, ser vi att andelen endast är """. Det finns trots allt ett stort antal olika yrken och en hel drös med utbildningar i Sverige, så även om datavetenskap och andra relaterande ämnen som statistik- eller teknik är populära, tar de ändå en förhållandevis liten plats i jämförelse med allt annat samlat. Dessutom finns givetvis chansen (eller risken, från denna rapports perspektiv) att vissa enskilda trådar äter upp ett stort antal inlägg som egentligen hade kunnat varit egna trådar.

Låt oss då se hur de ämnesrelaterade trådarna vi hittar i `filtered_df` har utvecklats under 2000-talet såhär långt. Som vi ser i följande kodstycke har jag filtrerat bort år 2021 eftersom det precis har startat. Det vore ju orättvist mot 2021 att inte ge det samma förutsättningar som de andra åren! 

In [6]:
# Diagram: Number of threads related to DS per year since 2003

Åren 2003 till 2007 har nästan noll till antalet. Det dröjer till 2008-2010 innan antalet börjar stiga på riktigt. Här passar det bra att tipsa den intresserade om en artikel från Forbes som heter "A Very Short History of Data Science" (länk i källförteckning). Artikeln är från 2013, men ger en informativ beskrivning av hur Data Science historia såg ut fram till den tidpunkten. I slutet av artikeln framgår det bland annat att tillväxten av analytiker- och Data Science-jobb i princip går spikrakt upp 2009-2010, vilket stämmer väl överens med vad figur 2 ovan antyder om intresset för ämnet. 

Efter 2012 ser vi däremot en oväntad nedgång i antalet inlägg. En tanke är att detta kan ha att göra med vad jag nämnde tidigare, nämligen att nya inlägg postas i befintliga trådar istället för att skapa nya trådar. Dessa kan alltså tänkas ta upp en hel del inlägg som egentligen skulle kunna ha varit starten på nya trådar.

När vi ändå är inne på det spåret kan vi också kolla på det datum då det senaste inlägget publicerades i de trådar med flest svar. Vi försöker alltså svara på frågan "Är de största trådarna fortfarande aktiva?". De 10 största trådarna från respektive kategori ser vi i de två tabellerna nedan.

In [7]:
# Table: Top 10 threads with the most answers from Utbildning och studier

# Table: Top 10 threads with the most answers from Arbetsliv och arbetsmarknad

"Master i statisik" innehåller 162 svar och därför skulle man kunna tänka sig denna innehåller många inlägg som annars kunde varit egna trådar. Denna tråds senaste inlägg publicerades dock 2010, så i det här fallet fallerade resonemanget tyvärr. I tabell 2 ser det bättre ut. Den tredje största tråden från *Arbetsmarknad och arbetsliv* heter nämligen "Data scientist", spot on med andra ord! Detta är en hyggligt stor tråd från 2017 med 126 svar, där det senaste svaret publicerades för två veckor sedan. Med andra ord tråd som fortfarande är aktiv. Även den fjärde största tråden är intressant, eftersom Business Intelligence är nära kopplat till Data Science. Denna tråd ser ut att vara ungefär lika aktiv som "Data scientist".

För att få en överblick över hur många relaterade trådar som verkar ha dött ut kollar vi även antalet trådar vars senaste inlägg publicerades samma år. 

In [8]:
# Diagram: Number of threads for which the latest answer were published the same year

Inte helt oväntat är det många trådar från 2011-2015 som inte är aktiva längre. Värt att notera är dock att det under perioden 2017-2020 är flera trådar som är aktiva än vad publicerats helt nya trådar. Det är givets vad man också kunde förväntat sig ju närmare nutid man tittar. En tråd som publicerades 2020 kan ju inte ha sitt senaste svar från 2019, till exempel. 

## Ordanalys

Vi ska nu se om vi kan ta reda på styrkan i inläggen. Med styrka menas alltså ordens "sentiment", som på engelska betyder ungefär "känsla" eller "stämning". För att lyckas med detta krävs en tabell med ett stort antal vanliga svenska ord tillsammans med respektive ords styrka. Ett sådant sentimentlexikon finns tillhandahållen på GitHub (se kod/källhänvisnning), fri att använda för vem som helst.

Innan vi tittar på inläggens styrka så undersöker vi vilka de vanligaste orden är. Ord som "jag", "hej" eller liknande kallas för stoppord. Dessa är förstås ointressanta. Istället skapar vi ett så kallat word cloud där dessa ord är bortfiltrerade. För detta ändamål används ytteliggare två paket; `wordcloud2` för skapandet av ordmolnet och `RColorBrewer` för tillhörande färgschema.

In [9]:
# Word cloud

Att visualisera de vanligaste orden på det här sättet är dels fint att titta på, men det är också ett effektivt sätt att bilda en uppfattning om de vanligaste orden. Matematik och programmering toppar listan.

Vidare till inläggens styrka. För att lyckas med detta skapar jag en ny data frame, `threads_sentiment_df`. Utöver de redan kända variablerna ID, title, date och categorie, innehåller denna tabell även

* *word_matches*: antal ord från trådstartarens första inlägg som matchats med något av orden i `sentiments_df`
* *post_strength*: den beräknade styrkan i respektive inlägg
* *pos_proportion* och *neg_proportion*: andelen matchade ord som är markerade som positiva respektive negativa.

Efter att ha sammanfogat de önskade variablerna (kolumnerna) från `filtered_df` med de önskade variablerna från `sentiments_df` passar jag även på att filtrera bort alla trådar som har färre än 20 stycken ordmatchningar. Anledningen till detta är att det finns trådar vars första inlägg är väldigt korta och kanske bara har ett fåtal ordmatchningar. Om dessa ord råkar vara markerade som negativa och med negativ styrka kommer hela inlägget att framstå som väldigt negativa, när deras enda brott kanske egentligen var att inlägget var kort. Kvar har vi en tabell med 126 trådar och följande struktur:

In [10]:
# Glimpse of sentiment data frame

Med hjälp av ett spridningsdiagram blir det enklare att förstå vår data. Jag låter en grön linje gå längs noll, som representerar en neutral styrkenivå. 

In [11]:
# Scatter diagram: Thread starters inital posts' word strength

Spridningen ser ganska jämn ut, möjligen något positivt lutande. Andelen positiva inlägg är `r prop_pos_posts*100`%. Däremot är den genomsnittliga styrkan i inläggen endast `r mean_strength`, dvs ganska neutral.

En sak som vore intressant i det här läget är att titta på de mest positiva och de mest negativa inläggen för att se om tabellen talar sanning. Följande inlägg har högst/lägst styrka.

In [12]:
# Table: Threads with highest strength

# Table: Threads with lowest strength

Öppningsinlägget från tråden "KTH eller Chalmers? Stockholmare som ska plugga datateknik." lyder såhär:

In [13]:
# Inital post from the thread with highest strength

Personen bakom inlägget använder ord som "bästa", "bättre", "passionerad", "lyckas" och "uppskattas" vilket antagligen lagt till positiv styrka från sentimenttabellen. Dock förekommer även negativa ord som "less", "rädd", "larm" och "jättefarligt", vilket såklart drar ner styrkan. 

Vi tittar också på öppningsinlägget från tråden "Stöd angående val av studier [Datavetenskap mm]".

In [14]:
# Inital post from the thread with lowest strength

Ja, ett inlägg som öppnar med "Jag har extrem ångest" är såklart illavarslande vad gäller den positiva styrkan. Sedan fortsätter inlägget med ord som till exempel "ifrågasätter", "misstag", "mesig", "dum" och "osäker". Reslutatet? Högsta placering på listan med de mest negativa inläggen!

# Slutsats

Vad gäller tesen om att antalet trådstartar skulle befinna sig i en slags uppåtlutande kurva måste jag tyvärr konstatera att inhämtad data inte styrker det påståendet. Snarare tvärtom faktiskt! Delförklaringen till detta kan vara att nya inlägg postas i befintliga trådar, men det kan likväl vara så att antalet aktiva användare på forumet har gått ned. 

Om både sentimentlistan och inläggen hade varit längre skulle antagligen tabellerna vara säkrare. Nu fanns det en stor begränsning i antal ord eftersom sentimentlistan inte täcker in i närheten av så många ord som man hade önskat (den består av 2067 ord). Dessutom innehåller inte inläggen så mycket text. Jag anser att sentimenttabellen tycks fungera någorlunda bra eftersom min känsla efter att ha läst inläggen korresponderar med inläggens påstådda styrka från tabellerna.

# Källförteckning

Press, G. 2013. *A Very Short History Of Data Science*. www.forbes.com. 2013-05-28. Tillgänglig: https://www.forbes.com/sites/gilpress/2013/05/28/a-very-short-history-of-data-science/?sh=21ae6fbb55cf [Hämtad 2021-01-13].

Sentimentlexikon från Språkbanken: Nusko, Bianka and Tahmasebi, Nina and Mogren, Olof. 2016. Building a Sentiment Lexicon for Swedish.